In [85]:
import os
import zipfile
import cdsapi
import xarray as xr

import numpy as np

from scipy.interpolate import RegularGridInterpolator
from scipy.optimize import brentq

from scipy.spatial.transform import Rotation

R = 8.314 #J/(mol*K)
M_air = 0.02896 #kg/mol

MAPEH1 = (22.600338,114.545072)
MAPEH2 = (22.608738,114.548871)
MAPEH3 = (22.614854,114.540537)

MAPzEH1 = -21.79
MAPzEH2 = 4.44
MAPzEH3 = 10.07

In [82]:
'''
指定数据的时间
'''
dayRequest = 15
monthRequest = 10
yearRequest = 2024

'''
指定数据的空间范围。根据测算的推荐值为N0-N40, E100-E130
'''
northBoundary = 40
southBoundary = 0
westBoundary  = 100
eastBoundary  = 130
areaRequest = [northBoundary,westBoundary,southBoundary,eastBoundary]

'''
给定数据的存储位置, zip文件保存在当前目录下/data/子目录，以时间命名。解压缩文件放在时间命名的文件夹下。
'''
dateStr = f'{yearRequest}{monthRequest}{dayRequest}'
currentPath = os.getcwd()
zipPath  = currentPath + f'/data/{dateStr}.zip'
dataPath = currentPath + f'/data/{dateStr}/'
geopDataPath = dataPath + 'geopotential_stream-oper_daily-mean.nc' # geop文件默认名称
tempDataPath = dataPath + 'temperature_0_daily-mean.nc'            # temp文件默认名称

'''
尝试打开指定当前路径下/data/{dateStr}/路径下的重力势数据和温度数据，否则下载新的文件
'''
try: 
    geopDataSet = xr.open_dataset(geopDataPath,engine='netcdf4')
    tempDataSet = xr.open_dataset(tempDataPath,engine='netcdf4')
except FileNotFoundError:
    dataset = "derived-era5-pressure-levels-daily-statistics"
    request = {
    "product_type": "reanalysis",
    "variable": [
        "geopotential",
        "temperature"
    ],
    "year": f"{yearRequest}",
    "month": [f"{monthRequest}"],
    "day": [f"{dayRequest}"],
    "pressure_level": [
        "1", "2", "3",
        "5", "7", "10",
        "20", "30", "50",
        "70", "100", "125",
        "150", "175", "200",
        "225", "250", "300",
        "350", "400", "450",
        "500", "550", "600",
        "650", "700", "750",
        "775", "800", "825",
        "850", "875", "900",
        "925", "950", "975",
        "1000"
    ],
    "daily_statistic": "daily_mean",
    "time_zone": "utc+08:00",
    "frequency": "6_hourly",
    "area": areaRequest
}
    client = cdsapi.Client()
    client.retrieve(dataset, request).download(zipPath)

    with zipfile.ZipFile(zipPath,'r') as z:
        fileList = z.namelist()
        print(f'{fileList}')
        os.mkdir(currentPath+f'/data/{dateStr}')
        z.extractall(currentPath+f'/data/{dateStr}/')
        geopDataSet = xr.open_dataset(currentPath+f'/data/{dateStr}/{fileList[0]}',engine='netcdf4')
        tempDataSet = xr.open_dataset(currentPath+f'/data/{dateStr}/{fileList[1]}',engine='netcdf4')

'''
如果是新下载的文件或者没有删除干净, 重新删除zip文件
'''
try:
    os.remove(zipPath)
except FileNotFoundError:pass

'''
获取数据集的各自变量和因变量, 将其化为SI单位制
'''
temperature = tempDataSet['t'].squeeze()
altitude = geopDataSet['z'].squeeze()/9.8066
pressure = temperature['pressure_level']*1e2 # pa
longitude = temperature['longitude']
latitude  = temperature['latitude']

density = pressure * M_air /(R * temperature)

tempInterp = RegularGridInterpolator((pressure,latitude,longitude),values=temperature)
densInterp = RegularGridInterpolator((pressure,latitude,longitude),values=density)
altiInterp = RegularGridInterpolator((pressure,latitude,longitude),values=altitude)

## 分层计算$\rho(X)$函数

### 1.对流层

对流层大气的高度和地形密切相关，高度大约为10km。而山体的高度在300m范围，如果不考虑山导致的大气深度变化，就会多考虑约3%的对流层大气。这是很重要的。

对流层大气密度大，一点小的大气密度变化就会导致倾斜深度x的绝对值的巨大变化。在**对流层集中了大气$75\%$的质量和几乎全部的水汽**。几乎所有的天气现象也集中在对流层。在对流层，高度每上升1km,温度降低约6.5K。

这里需要注意需不需要考虑云量。因为水蒸汽是18g/mol, 而干燥空气是28.97g/mol。所以混合了水汽的潮湿空气较轻，会上浮。上升后水汽遇冷，发生相变产生各种各样的云，其密度需要考虑干空气和水含量的影响，因此**总的密度会比干空气重一点**，但是又有下方上升气流的托举，所以云层可以存在。

|云的类型|主要成分|液态水含量|垂直尺度|备注|
|-----|-----|-----|-----|-----|
|卷云|冰晶|0.01-0.1 %  | 500 m| 稀薄云，密度接近晴空时的空气|
|层云|水滴|0.2-1 %   | 500m| 灰色云层，密度稍大|
|积云|水滴|0.1-2  %  | 1km | 蓬松的棉花状云，内部密度分布也不均匀，总体较大|
|积雨云|水滴/冰晶/过冷水|**10-25 %**|> 5km| 雷暴云，甚至可以托起冰雹|

对流层存在垂直方向的对流运动。
| 情况                | 温度变化 (K)    | 气压变化 (%) | 密度变化 (%) | 说明        |
| ----------------- | ----------- | -------- | -------- | --------- |
| 平均垂直对流（上升1 km）    | -9.8 K | -12%     | ≈ -15%   | 对流云区典型变化，这个是已经被考虑了的  |
| 冷暖空气平流（±10 K）     | ±10 K       | 稳定       | ±3.5%    | 冷锋/暖锋附近，这个给定了空间分布即可   |
| 湿度变化（RH从20%到100%） | —           | —        | -1～-2%   | 湿空气密度略减，和云层一起被考虑，具体依赖于云层覆盖率   |
| 下沉气流（2 km下沉）      | +20 K       | +20%     | +25%     | 高压区空气压缩变密 |

按照时间和空间的尺度

| 现象类型        | 典型空间尺度      | 典型时间尺度  | 对密度的影响  | 说明 |
| ----------- | ----------- | ------- | ------- | -----|
| 局地热对流（积云）   | 1–10 km     | 数分钟–数小时 | ±10–20% | 积云按照云层覆盖和比质量来算|
| 锋面系统（冷暖空气平流） | 100–1000 km | 数小时–数天  | ±5–15%  | 空间尺度过大，信息足够覆盖 |
| 高低压系统       | 500–3000 km | 数天      | ±5–10%  | 空间尺度过大，信息足够覆盖 |
| 平流层顶波动（重力波） | 10–1000 km  | 数分钟–数小时 | ±1–5%   | 时间尺度过小，日平均可覆盖 |

#### 1.1 非静态流体的密度变化

假设空气是理想气体并且流动是稳定的。流动是稳定的这一点其实可以保障，因为粒子穿过大气的时间非常短暂，远超过大气流动速度场变化的时间尺度。我们可以使用以下方法估算流动过程中的压强和密度变化。
用伯努利方程来计算压强变化。对于无粘、稳定的流动，伯努利方程为：
$$
p + \frac{1}{2} \rho v^2 = \text{Const}
$$

如果初始流速为0，初始压强为 ( $p_0 = 101325 Pa$ )，流速变为 ( $v = 10  m/s $ )，则在流动区域的新压强 ( p ) 可以通过伯努利方程求解：
$$
p + \frac{1}{2} \rho_0 v^2 = p_0
$$

$$
p \approx 101266.15 Pa
$$

密度的变化是由于流动速度的变化所引起的。温度保持不变，那么密度变化也可以通过状态方程来推算。

根据理想气体状态方程：
$$
\rho = \frac{p M}{R T}
$$

$$
\Delta \rho = \frac{\Delta p M}{R T}
$$

算出来密度变化大概**是$0.058\%$**

可以看出稳定流动的空气的压强和密度变化非常微小，可以认为在实际应用中变化不显著。


#### 1.2 对流层云量与湿度


In [83]:
a = np.arange(3)
np.sum(a**2)

np.int64(5)

In [247]:
altiInterp((0.8e5,*MAPEH1))
densInterp((0.8e5,*MAPEH1))

def ERA5h2rho(h_m,loc,altiInterp,densInterp):
    equation = lambda p : (float)(altiInterp((p,*loc))) - h_m
    rootp = brentq(equation,a = pressure.min(),b = pressure.max())
    return (float)(densInterp((rootp,*loc)))

def locGeneratorL(L_m,ztheta,zphi,loc0):
    '''
    定义地球半径
    '''
    Rearth = 6371*1e3
    '''
    将经纬度换算为赤道坐标系下的球坐标
    '''
    ltheta = np.deg2rad(90.-loc0[0])
    lphi = np.deg2rad(loc0[1])
    '''
    计算经度为0时地平坐标系基矢量在赤道坐标系下的表达
    '''
    v3n = np.array([-np.cos(ltheta),0,np.sin(ltheta)])
    v3e = np.array([0,1.,0])
    v3z = np.array([np.sin(ltheta),0,np.cos(ltheta)])
    '''
    组合基矢量成为旋转矩阵，这个肯定是归一化的。注意是列向量所以得转置。
    '''
    R1 = np.array([v3e,v3n,v3z]).T
    '''
    调用scipy.spatial.transform.Rotation, 生成绕地轴旋转的矩阵。
    '''
    R2 = Rotation.from_euler('z',lphi)
    '''
    计算在地平坐标系下的x长度的直角坐标系下的矢量
    '''
    v3vec = L_m * np.array([np.sin(np.deg2rad(ztheta))*np.cos(np.deg2rad(zphi)),
                            np.sin(np.deg2rad(ztheta))*np.sin(np.deg2rad(zphi)),
                            np.cos(np.deg2rad(ztheta))])
    '''
    执行地平坐标系到赤道坐标系的转换
    '''
    v3tran = R2.apply(R1@v3vec)
    print(v3tran)
    '''
    计算在赤道坐标系下的位置矢量
    '''
    v3loc = Rearth *np.array([  np.sin(ltheta)*np.cos(lphi),
                                np.sin(ltheta)*np.sin(lphi),
                                np.cos(ltheta)])
    '''
    赤道坐标系下x长度向量与位置向量的向量和, 为x长度下赤道坐标系的绝对坐标
    '''
    v3tol = (v3loc+v3tran)
    '''
    计算x长度向量投影到球表面到表面的高度。
    '''
    h = np.sqrt(np.sum(v3tol**2)) - Rearth
    '''
    计算x长度向量投影到球表面到表面的坐标。
    '''
    latitude = 90. - np.rad2deg(np.arctan(np.sqrt(v3tol[0]**2+v3tol[1]**2)/v3tol[2]))
    longitude = np.rad2deg(np.arccos(v3tol[0]/np.sqrt(v3tol[0]**2+v3tol[1]**2)))

    return (latitude,longitude),h


ERA5h2rho(1e4,MAPEH1,altiInterp,densInterp)
print(MAPEH1)
locGeneratorL(100*1e3,90,180,MAPEH1)

(22.600338, 114.545072)
[9.09634769e+04 4.15408940e+04 1.36592001e-11]


((np.float64(22.597400592008256), np.float64(113.57103904381817)),
 np.float64(784.7578209983185))

## 继承MCEq的GeneralizedTarget类

In [ ]:
from MCEq.geometry.density_profiles import GeneralizedTarget

class ERA5model(GeneralizedTarget):
    def __init__(self):
        print("调用子类构造方法")
 
    def childMethod(self):
        print('调用子类方法')

era5test = ERA5model()
#era5test.print_table()

调用子类构造方法
